# Parallelized version

In [1]:
import math
import random
import numpy as np
import time

## Initializing spark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("appName").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [4]:
sc.getConf().getAll()

[('spark.app.id', 'local-1669673861382'),
 ('spark.driver.port', '39653'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'appName'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.ui.showConsoleProgress', 'true')]

## Functions

### Auxiliar functions

In [5]:
def trainTestSplit(dataset):
    train = dataset.filter(lambda x: x[2]==1)
    train = train.map(lambda x: (x[0], x[1]))
    test = dataset.filter(lambda x: x[2]==0)
    test = test.map(lambda x: (x[0], x[1]))
    return (train, test)

In [6]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [7]:
def rdd_cost_function(RDD_Xyyhat, lambda_ref, m, w):
    y = RDD_Xyyhat[1]
    y_hat = RDD_Xyyhat[2]
    temp = (-1/m) * (
        y * np.log(y_hat) + (1 - y)
        * np.log(1 - y_hat)
    )
    cost = temp + (lambda_ref/(2*m))*sum([i*i for i in w.value])
    return cost

In [8]:
def str_to_number_list(line):
    numbers = line.replace(" ","").split(",")
    numbers = [float(number) for number in numbers]
    
    # Separate X and y
    numbers = (numbers[:-1], numbers[-1])
    
    return numbers

In [9]:
def sum_list_values(list1, list2):
    return [n1 + n2 for n1, n2 in zip(list1, list2)]

In [10]:
def sum_minus_mean_squared(numbers):
    return [(number-means.value[x])**2 for x,number in enumerate(numbers)]

In [11]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [12]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [13]:
def rdd_std_by_column(RDD_Xy, m):
    stds = RDD_Xy.map(lambda x: x[0]).map(sum_minus_mean_squared).reduce(sum_list_values)
    stds = [math.sqrt(x/m) for x in stds]
    return stds

In [14]:
def multiply_RDDXy_by_w(Xy):
    global b
    tot = 0
    for xi, wi in zip(Xy[0], w.value):
        tot += xi * wi
    tot += b
    return (Xy[0], Xy[1], sigmoid(tot))

In [15]:
def calculate_dw(RDD_Xyyhat):
    dw = []
    for x in RDD_Xyyhat[0]:
        dw.append((RDD_Xyyhat[2]-RDD_Xyyhat[1])*x)
    return dw

### Mandatory functions

In [16]:
def readFile(filename):
    dataset = sc.textFile(filename)
    dataset = dataset.map(str_to_number_list)
    return dataset.sample(False, 1)

In [17]:
def normalize(RDD_Xy):
    m = RDD_Xy.count()
    global means
    means = sc.broadcast(rdd_mean_by_column(RDD_Xy, m))
    stds = sc.broadcast(rdd_std_by_column(RDD_Xy, m))
    norm_rdd = RDD_Xy.map(lambda values: ([(x - mean) / std for x, mean, std in zip(values[0], means.value, stds.value)], values[1]))
    return norm_rdd

In [18]:
def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    global b
    global m
    m = RDD_Xy.count()
    n = len(RDD_Xy.take(1)[0][0])
    global w
    w_temp = w.value.copy()
    for it in range(iterations):
        RDD_Xyyhat = RDD_Xy.map(multiply_RDDXy_by_w)
        print(f"Cost for it {it}:", RDD_Xyyhat.map(lambda x: rdd_cost_function(x, lambda_reg, m, w)).reduce(lambda x,y: x+y))
        dw=[0 for i in range(0,n)]
        for cl in range(n):
                X_cl = RDD_Xyyhat.map(lambda x: (x[2]-x[1])*x[0][cl]).reduce(lambda x, y: x+y)
                dw[cl] = (1/m)*(X_cl)+(lambda_reg/m)*w_temp[cl]
                w_temp[cl] -= learning_rate * dw[cl]
        w = sc.broadcast(w_temp)
        db = (1/m)*RDD_Xyyhat.map(lambda x: x[2]-x[1]).reduce(lambda x, y: x+y)
        b -= learning_rate * db
    return w, b

In [28]:
def checkPrediction(y, y_hat):
    if (y == y_hat and y_hat==0):
        return (0, 1, 0, 0)
    if (y == y_hat and y_hat==1):
        return (1, 0, 0, 0)
    if (y != y_hat and y_hat==1):
        return (0, 0, 1, 0)
    if (y != y_hat and y_hat==0):
        return (0,0,0,1)

In [71]:
def accuracy(ws, b, RDD_Xy):
    total = RDD_Xy.count()
    y_and_y_hat = RDD_Xy.map(lambda x: (x[1], predict(ws, x[0], b)))
    result = y_and_y_hat.map(lambda x: checkPrediction(x[0], x[1]))
    tp = result.map(lambda x: x[0]).reduce(lambda x,y: x+y)
    tn = result.map(lambda x: x[1]).reduce(lambda x,y: x+y)
    fp = result.map(lambda x: x[2]).reduce(lambda x,y: x+y)
    fn = result.map(lambda x: x[3]).reduce(lambda x,y: x+y)
    acc = (tp+tn)/total
    return acc, tp, tn, fp, fn

In [56]:
def predict(w, X, b):
    tot = 0
    for xi, wi in zip(X, w.value):
        tot += xi * wi
    tot += b
    val = sigmoid(tot)
    if (val>=0.5):
        return 1.0
    return 0.0

## Testing

In [22]:
RDD_Xy = readFile("../data/botnet_tot_syn_l.csv")
RDD_Xy.count()

1000000

In [23]:
RDD_Xy_normalized = normalize(RDD_Xy)

In [24]:
RDD_Xy_presplit = RDD_Xy_normalized.map(lambda x: (x[0], x[1], 1 if random.random() < 0.75 else 0))
train_data, test_data = trainTestSplit(RDD_Xy_presplit)

#### Initialize some stuff before training

In [25]:
np.random.seed(33)

global b  
b = 0

global n
n = len(RDD_Xy.take(1)[0][0])

global w
w = sc.broadcast(np.random.rand(n))

#### Training

In [26]:
w_final, b = train(train_data, 10, 1.5, 0)

Cost for it 0: 1.4449531970006226
Cost for it 1: 0.736083239170125
Cost for it 2: 0.45035464825842064
Cost for it 3: 0.34626879593069926
Cost for it 4: 0.29734168151745477
Cost for it 5: 0.26988381803261
Cost for it 6: 0.2518216159193672
Cost for it 7: 0.2389376063162097
Cost for it 8: 0.2292108960597853
Cost for it 9: 0.22182164501341034


#### Accuracy

In [70]:
acc, tp, tn, fp, fn = accuracy(w_final, b, test_data)

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = (2*precision*recall)/(precision+recall)
total = tp+tn+fp+fn
# print(tp/total)
# print(tn/total)
# print(fp/total)
# print(fn/total)
# print("____________________")
# print(precision)
# print(recall)
# print(f1)

(1, 0, 0, 0)


## Experiments

In [38]:
elapsed_times = {}

for cores in range(1,13,1):
    sc.stop()
    conf = SparkConf().setAppName("appName").setMaster(f"local[{cores}]")
    sc = SparkContext(conf=conf)
    
    print(f"---------- Starting execution with {cores} cores ----------")
    
    
    # ---------- Execution ----------
    RDD_Xy = readFile("../data/botnet_tot_syn_l.csv")
    RDD_Xy_normalized = normalize(RDD_Xy)
    RDD_Xy_presplit = RDD_Xy_normalized.map(lambda x: (x[0], x[1], 1 if random.random() < 0.75 else 0))
    train_data, test_data = trainTestSplit(RDD_Xy_presplit)
    start = time.time()
    
    np.random.seed(33)
    global b  
    b = 0
    global n
    n = len(RDD_Xy.take(1)[0][0])
    global w
    w = sc.broadcast(np.random.rand(n))
    w_final, b = train(train_data, 10, 1.5, 0)
    acc, tp, tn, fp, fn = accuracy(w_final, b, test_data)
    print(f"The accuracy for the test set is: {acc}")
    total = tp+tn+fp+fn
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = (2*precision*recall)/(precision+recall)
    
    print(f"The accuracy for the test set is: {acc}")
    print(f"The tp percentage is: {tp/total}")
    print(f"The tn percentage is: {tn/total}")
    print(f"The fp percentage is: {fp/total}")
    print(f"The fn percentage is: {fn/total}")
    print(f"The precision is: {precision}")
    print(f"The recall is: {recall}")
    print(f"The f1 score is: {f1}")
    # ---------- Execution ----------

    end = time.time()
    print(f"---------- Finished execution with {cores} cores ----------")
    elapsed_time = end - start
    elapsed_times[cores] = elapsed_time
    print(f"Elapsed time for cores {cores} is {elapsed_time} seconds")

---------- Starting execution with 1 cores ----------
Cost for it 0: 1.4435202880096918
Cost for it 1: 0.7356552805180301
Cost for it 2: 0.45015531164469985
Cost for it 3: 0.3456708846232785
Cost for it 4: 0.29697862064727
Cost for it 5: 0.2695643477219923
Cost for it 6: 0.2521515823751659
Cost for it 7: 0.238485670267591
Cost for it 8: 0.2291371233147315
Cost for it 9: 0.22152240030609552
[(1, 0, 0, 0)]
The accuracy for the test set is: 0.929277980216554
The accuracy for the test set is: 0.929277980216554
The tp percentage is: 0.47250699720111955
The tn percentage is: 0.4564254298280688
The fp percentage is: 0.04313874450219912
The fn percentage is: 0.027928828468612554
The precision is: 0.916340345680544
The recall is: 0.9441909890619132
The f1 score is: 0.9300572166124932
---------- Finished execution with 1 cores ----------
Elapsed time for cores 1 is 1831.2043297290802 seconds
---------- Starting execution with 2 cores ----------
Cost for it 0: 1.4457456026235944
Cost for it 1: 0.

Cost for it 2: 0.4506816532006873
Cost for it 3: 0.34532073985710665
Cost for it 4: 0.2972105100532544
Cost for it 5: 0.2691873360321746
Cost for it 6: 0.25155942205089865
Cost for it 7: 0.2388412838823025
Cost for it 8: 0.2286747993624267
Cost for it 9: 0.2218180399214867
[(0, 1, 0, 0)]
The accuracy for the test set is: 0.9253963556081719
The accuracy for the test set is: 0.9253963556081719
The tp percentage is: 0.4725696435517309
The tn percentage is: 0.4552424642208383
The fp percentage is: 0.044097891946731256
The fn percentage is: 0.028090000280699502
The precision is: 0.9146493849198649
The recall is: 0.9438940193667753
The f1 score is: 0.9290416164100623
---------- Finished execution with 10 cores ----------
Elapsed time for cores 10 is 415.4833137989044 seconds
---------- Starting execution with 11 cores ----------
Cost for it 0: 1.4454179711607393
Cost for it 1: 0.7364736864538415
Cost for it 2: 0.450753926765049
Cost for it 3: 0.34590105733409
Cost for it 4: 0.297067365284884

In [39]:
elapsed_times

{1: 1831.2043297290802,
 2: 965.405748128891,
 3: 706.283593416214,
 4: 562.6039688587189,
 5: 505.5888659954071,
 6: 498.65091252326965,
 7: 501.611444234848,
 8: 398.59453678131104,
 9: 408.00764179229736,
 10: 415.4833137989044,
 11: 440.38226985931396,
 12: 428.98688793182373}